In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import unidecode
import dateparser

tqdm.pandas()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

import constants

| Kategori                                                               | İçerik Sayısı     | Hedef Değişkenler                                                                                            |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`,`mevzuat_tarihi`, `madde_sayisi`
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`
|
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | `kategori`, `donem` ve `sira_no`                                                                       |
| [Genelge](#kategori-genelge)                                           | 385               | `kategori`, `mevzuat_tarihi` ve varsa `mevzuat_no` yoksa `belge_sayi` 
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `madde_sayisi` ve varsa ` mevzuat_tarihi`
| [Tüzük](#kategori-tüzük)                                               | 82                |  `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum`, `madde_sayisi`       |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum` 
| [Özelge](#kategori-özelge)                                             | 750               | `kategori`, `mevzuat_tarihi`

In [4]:
df=pd.read_csv('../data/kanunum-nlp-doc-analysis-dataset.csv')

In [5]:
kanun=df[df.kategori=='Kanun']
khk=df[df.kategori=='Kanun Hükmünde Kararname']
resmi_gazete=df[df.kategori=='Resmi Gazete']
genelge=df[df.kategori=='Genelge']
c_kararname=df[df.kategori=='Cumhurbaşkanlığı Kararnamesi']
tüzük=df[df.kategori=='Tüzük']
yönetmelik=df[df.kategori=='Yönetmelik']
teblig=df[df.kategori=='Tebliğ']
ozelge=df[df.kategori=='Özelge']

In [51]:
dateparser.parse('18kasim1992')

In [46]:
dateparser.parse('18aralık1992')

datetime.datetime(1992, 12, 18, 0, 0)

In [56]:
class FeatureExtractor:
    def __init__(self):
        self.feature_name = None
        self.cols = []

    def _extractor_func(self, row_data):
        raise NotImplementedError()

    def extract(self, input_df):
        print(f"'{self.feature_name}' cikariliyor... -> '{self.feature_name}_pred'")
        feature_indexer = input_df.kategori.isin(self.cols)
        input_df[self.feature_name + '_pred'] = None
        input_df.loc[feature_indexer,
                     self.feature_name + '_pred'] = input_df[feature_indexer].progress_apply(
            lambda x: self._extractor_func(x),
            axis=1)

class RegaTarihiExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "rega_tarihi"
        self.cols = constants.COLS_REGA_TARIHI
    
    def tarih_duzelt(self,text):
        text = text.replace('\n', '')
        text = text.replace('/n', '')
        text = text.replace(' ', '')
        #text = text.replace('  ', '')
        text = text.replace('î', 'i')
        text = text.replace('nısan', 'nisan')
        text = text.replace('mayis', 'mayıs')
        text = text.replace('ekım', 'ekim')
        text = text.replace('kasim', 'kasım')
        text = text.replace('aralik', 'aralık')
        return text

    def _extractor_func(self, row_data):
        NoneType = type(None)
        #std_txt = unidecode.unidecode(row_data.data_text).lower()
        rega_tarihi = np.nan
        
        try:
            std_txt = unidecode.unidecode(row_data.data_text).lower()
            if row_data.kategori == 'Kanun':
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
            #######################################################
            elif row_data.kategori == 'Kanun Hükmünde Kararname':
                rega_tarihi =  std_txt.split("resmi gazete tarihi: ", 1)[1].split('resmi gazete sayisi:', 1)[0]
            #######################################################
            elif row_data.kategori == 'Resmi Gazete':
                std_txt = unidecode.unidecode(row_data.data_text).lower()
                a=std_txt.split('sayi')[0].replace(' ','').split('\n')
                while '' in a:
                    a.remove('')
                date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-2]),languages=['tr']),dayfirst=True)
                if type(date2) == NoneType:
                    date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-1]),languages=['tr']),dayfirst=True)
                    if type(date2) == NoneType:
                        date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-3]),languages=['tr']),dayfirst=True)
                        if type(date2) == NoneType:
                            date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-4]),languages=['tr']),dayfirst=True)
                            if type(date2) == NoneType:
                                date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-5]),languages=['tr']),dayfirst=True)
                                
                                if type(date2) == NoneType:
                                    a=std_txt.replace(' ','').split('sayi')[0].replace(' ','').split('\n')
                                    while '' in a:
                                        a.remove('')
                                    date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-2]),languages=['tr']),dayfirst=True)
                                    if type(date2) == NoneType:
                                        date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-1]),languages=['tr']),dayfirst=True)
                                        if type(date2) == NoneType:
                                            date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a[-3]),languages=['tr']),dayfirst=True)
                                            if type(date2) == NoneType:
                                                a=''.join(std_txt.lower().split('sayı')[0].split(' ')[-4:])
                                                ''.join(std_txt.lower().split('sayı')[0].split(' ')[-4:])
                                                date2=pd.to_datetime(dateparser.parse(self.tarih_duzelt(a),languages=['tr']),dayfirst=True)
                                                if type(date2) == NoneType:
                                                    date2=np.nan

                rega_tarihi=date2
            #######################################################
            elif row_data.kategori == "Cumhurbaşkanlığı Kararnamesi":
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0].replace(" ","")
            #######################################################
            elif row_data.kategori == 'Tüzük':  
                rega_tarihi=std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0].replace(" ","")
            #######################################################
            elif row_data.kategori == 'Yönetmelik':
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
            #######################################################
            elif row_data.kategori == 'Tebliğ':
                rega_tarihi = std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[0]
        except:
            rega_tarihi=np.nan
        try:
            rega_tarihi=str(pd.to_datetime(rega_tarihi,dayfirst=True).date())
        except:
            rega_tarihi=np.nan
        return rega_tarihi
    

class MevzuatTarihiExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "mevzuat_tarihi"
        self.cols = constants.COLS_MEVZUAT_TARIHI
    
    def tarih_duzelt(self,text):
        text = text.replace('nısan', 'nisan')
        text = text.replace('mayis', 'mayıs')
        text = text.replace('ekım', 'ekim')
        text = text.replace('kasim', 'kasım')
        text = text.replace('\n', '')
        text = text.replace('/n', '')
        #text = text.replace(' ', '')
        return text

    def _extractor_func(self, row_data):
        #std_txt = unidecode.unidecode(row_data.data_text).lower()
        std_txt = row_data.data_text.lower()
        mevzuat_tarihi = np.nan
        try:
            if row_data.kategori == 'Kanun':
                std_txt = unidecode.unidecode(row_data.data_text).lower()
                mevzuat_tarihi = pd.to_datetime(std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[1]\
                                                    .split('kabul tarihi : ')[1].split('\n')[0],dayfirst=True)

            #######################################################
            elif row_data.kategori == 'Kanun Hükmünde Kararname':
                std_txt = unidecode.unidecode(row_data.data_text).lower()
                mevzuat_tarihi = pd.to_datetime(std_txt.split("resmi gazete tarihi: ",1)[1].split('resmi gazete sayisi:',1)[1]\
                                                    .split('kararnamenin tarihi : ')[1].split('\n')[0],dayfirst=True)
            #######################################################
            elif row_data.kategori == 'Genelge':
                try:
                    std_txt=row_data.data_text
                    std_txt=std_txt.split('Tarih',1)[1].split('Sayı')[0].replace('\n','')
                    mevzuat_tarihi=pd.to_datetime(std_txt,dayfirst=True)
                except:
                    try:
                        std_txt=row_data.data_text
                        mevzuat_tarihi=pd.to_datetime(std_txt.split('Sayı',1)[0].split('Tarih')[1].replace('\n',''),dayfirst=True)
                    except:
                        try:
                            mevzuat_tarihi=pd.to_datetime(re.findall(r'\d{2}.\d{2}.\d{4}', std_txt.replace('-',' ')\
                                                                     .replace('/',''))[0],dayfirst=True)
                        except:
                            try:
                                mevzuat_tarihi=pd.to_datetime(re.findall(r'\d{2}/\d{2}/\d{4}', std_txt.replace('-',' '))[0],dayfirst=True)
                            except:
                                try:
                                    mevzuat_tarihi=pd.to_datetime(re.findall(r'\d{1}/\d{1}/\d{4}', std_txt)[0],dayfirst=True)
                                except:
                                    mevzuat_tarihi=np.nan
            #######################################################
            elif row_data.kategori == 'Cumhurbaşkanlığı Kararnamesi':
                mevzuat_tarihi = self.tarih_duzelt(row_data.data_text.split('\n')[-1])
                mevzuat_tarihi = pd.to_datetime(dateparser.parse(mevzuat_tarihi,languages=['tr']),dayfirst=True)
            #######################################################
            elif row_data.kategori == 'Tüzük':
                mevzuat_tarihi = pd.to_datetime(std_txt.split("bkk no: ",1)[0].split("karar tarihi: ",1)[1].replace(",","")\
                                                .replace(" ",""),dayfirst=True)
            #######################################################
            elif row_data.kategori == 'Özelge':
                mevzuat_tarihi = std_txt.split("konu",1)[0].split("\n\n")
                try:
                    while '' in mevzuat_tarihi:
                        mevzuat_tarihi.remove('')
                    try:
                        mevzuat_tarihi=pd.to_datetime(mevzuat_tarihi[-1],dayfirst=True)
                    except:
                        try:
                            mevzuat_tarihi=pd.to_datetime(std_txt.split("tarih",1)[1].split('sayi')[0],dayfirst=True)
                        except:
                            try:
                                mevzuat_tarihi=pd.to_datetime(std_txt.split("sayi",1)[0].split('tarih')[1]\
                                                              .replace('\n','').replace(" ","").replace(":",""),dayfirst=True)
                            except:
                                try:
                                    mevzuat_tarihi=pd.to_datetime(std_txt.split('sayi')[0].split('tarih')[-1],dayfirst=True)
                                except:
                                    try:
                                        mevzuat_tarihi=pd.to_datetime(std_txt.split('*')[0].split(' ')[-1],dayfirst=True)
                                    except:
                                        try:
                                            mevzuat_tarihi=std_txt.split('konu')[0].replace('\n',"").split(' ')
                                            while '' in mevzuat_tarihi:
                                                mevzuat_tarihi.remove('')
                                            mevzuat_tarihi=pd.to_datetime(mevzuat_tarihi[-1],dayfirst=True)
                                        except:
                                            try:
                                                mevzuat_tarihi=pd.to_datetime(re.search(r'\d{2}.\d{2}.\d{4}', std_txt)\
                                                                              .group(),dayfirst=True)
                                            except:
                                                try:
                                                    mevzuat_tarihi=pd.to_datetime(re.search(r'\d{2}/\d{2}/\d{4}', std_txt)\
                                                                                  .group(),dayfirst=True)
                                                except:
                                                    mevzuat_tarihi=np.nan        
                
                except:
                    pass
        except:
            pass
        try:
            mevzuat_tarihi=str(mevzuat_tarihi.date())
        except:
            mevzuat_tarihi=np.nan
            
        return mevzuat_tarihi

In [57]:
rega_tarihi_extractor = RegaTarihiExtractor()
mevzuat_tarihi_extractor = MevzuatTarihiExtractor()

In [58]:
rega_tarihi_extractor.extract(df)

'rega_tarihi' cikariliyor... -> 'rega_tarihi_pred'


100%|█████████████████████████████████████████████████████████████████████████████| 2507/2507 [00:14<00:00, 178.11it/s]


In [9]:
mevzuat_tarihi_extractor.extract(df)

'mevzuat_tarihi' cikariliyor... -> 'mevzuat_tarihi_pred'


100%|█████████████████████████████████████████████████████████████████████████████| 1876/1876 [00:07<00:00, 236.78it/s]


In [59]:
#from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=rega_tarihi_extractor,
                   include_na=True
                  )

Veri Sayisi: 2507
Dogru Tahmin Sayisi: 2333
Yanlis Tahmin Sayisi: 174
Basari Orani: 0.9306


In [19]:
#from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=mevzuat_tarihi_extractor,
                   include_na=True
                  )

Veri Sayisi: 1876
Dogru Tahmin Sayisi: 1730
Yanlis Tahmin Sayisi: 146
Basari Orani: 0.9222


In [100]:
#df[(df.kategori=='Resmi Gazete')&(df.rega_tarihi_pred.isnull())]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4142 entries, 0 to 4141
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   4142 non-null   int64  
 1   kategori             4142 non-null   object 
 2   baslik               4142 non-null   object 
 3   rega_no              2527 non-null   object 
 4   mukerrer_no          3642 non-null   float64
 5   rega_tarihi          2527 non-null   object 
 6   kurum                2488 non-null   object 
 7   mevzuat_no           1091 non-null   object 
 8   belge_sayi           43 non-null     object 
 9   mevzuat_tarihi       1905 non-null   object 
 10  donem                500 non-null    object 
 11  sira_no              500 non-null    object 
 12  madde_sayisi         4142 non-null   int64  
 13  data_text            4142 non-null   object 
 14  url                  4142 non-null   object 
 15  kanunum_url          4142 non-null   o

In [28]:
df[(df.kategori=='Resmi Gazete')&(df.rega_tarihi_pred.isnull())]

,id,kategori,baslik,rega_no,mukerrer_no,rega_tarihi,kurum,mevzuat_no,belge_sayi,mevzuat_tarihi,donem,sira_no,madde_sayisi,data_text,url,kanunum_url,rega_tarihi_pred,mevzuat_tarihi_pred
532,1079,Resmi Gazete,10.06.1981 tarihli 17366 sayılı Resmi Gazete,17366,0.0,1981-06-10,NaN,NaN,NaN,NaN,NaN,NaN,0,T.C. \nResmî Gazete \n\nKuruluş Tarihi: (7 Teş...,http://www.resmigazete.gov.tr/arsiv/17366.pdf,https://www.kanunum.com/content/672778,NaN,NaN
533,1080,Resmi Gazete,18.08.1983 tarihli 18139 sayılı Resmi Gazete,18139,0.0,1983-08-18,NaN,NaN,NaN,NaN,NaN,NaN,0,T . C . \nResmî Gazete \n\nKuruluş Tar ihi : (...,http://www.resmigazete.gov.tr/arsiv/18139.pdf,https://www.kanunum.com/content/674384,NaN,NaN
534,1081,Resmi Gazete,16.12.1986 tarihli 19313 sayılı Resmi Gazete,19313,0.0,1986-12-16,NaN,NaN,NaN,NaN,NaN,NaN,0,T.C. \nResmî Gazete \n\nKuruluş Tarihi : ( 7 T...,http://www.resmigazete.gov.tr/arsiv/19313.pdf,https://www.kanunum.com/content/676878,NaN,NaN
538,1090,Resmi Gazete,22.05.1991 tarihli 20878 sayılı Resmi Gazete 1...,20878,1.0,1991-05-22,NaN,NaN,NaN,NaN,NaN,NaN,0,Başbakanlık \nMevzuatı Geliştirme ve Yayın Gen...,http://www.resmigazete.gov.tr/arsiv/20878_1.pdf,https://www.kanunum.com/content/706632,NaN,NaN
558,1123,Resmi Gazete,25.11.1931 tarihli 1959 sayılı Resmi Gazete,1959,0.0,1931-11-25,NaN,NaN,NaN,NaN,NaN,NaN,0,T.C. \nTesis tarihi: 7 Teşrinievvel 1336 \n\nK...,http://www.resmigazete.gov.tr/arsiv/1959.pdf,https://www.kanunum.com/content/677586,NaN,NaN
581,1165,Resmi Gazete,07.01.1982 tarihli 17567 sayılı Resmi Gazete,17567,0.0,1982-01-07,NaN,NaN,NaN,NaN,NaN,NaN,0,T.C. \nResmi Gazete \n\nKuruluş Tarihi: (7 Teş...,http://www.resmigazete.gov.tr/arsiv/17567.pdf,https://www.kanunum.com/content/673194,NaN,NaN
586,1178,Resmi Gazete,12.05.1961 tarihli 10805 sayılı Resmi Gazete,10805,0.0,1961-05-12,NaN,NaN,NaN,NaN,NaN,NaN,0,T . C \nResmi Gazete \n\nKuruluş tarihi: 7 E k...,http://www.resmigazete.gov.tr/arsiv/10805.pdf,https://www.kanunum.com/content/652894,NaN,NaN
620,1250,Resmi Gazete,07.01.1930 tarihli 1391 sayılı Resmi Gazete,1391,0.0,1930-01-07,NaN,NaN,NaN,NaN,NaN,NaN,0,T.C. \nGazete \n\nTesis tarihi: 7 Teşrinievvel...,http://www.resmigazete.gov.tr/arsiv/1391.pdf,https://www.kanunum.com/content/665022,NaN,NaN
623,1253,Resmi Gazete,02.03.1962 tarihli 11049 sayılı Resmi Gazete,11049,0.0,1962-03-02,NaN,NaN,NaN,NaN,NaN,NaN,0,26/2/1962 \n\n1928 - 1960 Bütçe yıllarına ait ...,http://www.resmigazete.gov.tr/arsiv/11049.pdf,https://www.kanunum.com/content/653418,NaN,NaN
628,1262,Resmi Gazete,16.11.1981 tarihli 17516 sayılı Resmi Gazete,17516,0.0,1981-11-16,NaN,NaN,NaN,NaN,NaN,NaN,0,T.C. \nResmî Gazete \n\nKuruluş Tarihi: (7 Teş...,http://www.resmigazete.gov.tr/arsiv/17516.pdf,https://www.kanunum.com/content/673092,NaN,NaN


In [35]:
resmi_gazete.data_text[534]#.lower().replace(' ','')

'T.C. \nResmî Gazete \n\nKuruluş Tarihi : ( 7 Teşrinievvel 1336 ) — 7 Ekim 1920 \n\nYönetim ve yazı işleri için 16 Aralık 1986 Başbakanlık Mevzuatı Geliştirme ve 16 Aralık 1986 Sayı : 19313 Yayın Genel Müdürlüğüne SALI Sayı : 19313 başvurulur. SALI \n\nYÜRÜTME VE İDARE BÖLÜMÜ \n\nBakanlar Kurulu Kararları \nKarar Sayısı 86/11226 \n\nE k l i "Kimyevi Gübrelerin İ thalinde Alınacak Fonlar ile Tar ımda Kullanılan Kimyevi \nGübrelerin Yurt İçinden ve Yurt Dışından Tedariki, Dağıtımı ve Desteklenmesi Hakkında Ka -\nrar \' ın 2 nci Maddesine Bir Fıkra Eklenmesine Dair Karar" ın yürürlüğe konulması; 20/2/1930 \ntarihli ve 1567 sayılı Kanun ile 2/2/1984 tarihli ve 2976 sayılı Kanun hükümlerine göre, Ba-\nkanlar Kurulu \'nca 17/11/1986 tarihinde kararlaştırılmıştır. \n\nT ÖZAL \nBaşbakan \n\nI K ERDEM \nDevlet Bak -Başbakan Yrd \n\nA TENEKECİ \nDevlet Bakanı \n\nM O SUNGURLU \nAdalet Bakanı \n\nA K ALPTEMOÇIN \nMaliye ve Gümrük Bakanı \n\nV ATASOY \nUlaştırma Bakanı \n\nK OKSAY \nDevlet Bakanı 

In [37]:
resmi_gazete.data_text[534].lower().split('sayı')[0].split(' ')

['t.c.',
 '\nresmî',
 'gazete',
 '\n\nkuruluş',
 'tarihi',
 ':',
 '(',
 '7',
 'teşrinievvel',
 '1336',
 ')',
 '—',
 '7',
 'ekim',
 '1920',
 '\n\nyönetim',
 've',
 'yazı',
 'işleri',
 'için',
 '16',
 'aralık',
 '1986',
 'başbakanlık',
 'mevzuatı',
 'geliştirme',
 've',
 '16',
 'aralık',
 '1986',
 '']

In [42]:
''.join(resmi_gazete.data_text[534].lower().split('sayı')[0].split(' ')[-4:])

'16aralık1986'

In [80]:
a=resmi_gazete.data_text[611].lower().split('sayı')[0].replace(' ','').split('\n')
while '' in a:
    a.remove('')
date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-2]),languages=['tr']),dayfirst=True)
if type(date2) == NoneType:
    date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-1]),languages=['tr']),dayfirst=True)
    if type(date2) == NoneType:
        date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-3]),languages=['tr']),dayfirst=True)
        if type(date2) == NoneType:
            date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-4]),languages=['tr']),dayfirst=True)
            if type(date2) == NoneType:
                date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-5]),languages=['tr']),dayfirst=True)
                if type(date2) == NoneType:
                    date2=np.nan
rega_tarihi=date2
print(rega_tarihi)
    

nan


In [96]:
a=resmi_gazete.data_text[611].lower().split('sayı')[0].split(' ')
a=''.join(a[-4:])
date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a),languages=['tr']),dayfirst=True)
#if type(date2) == NoneType:
#    date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-1]),languages=['tr']),dayfirst=True)
#    if type(date2) == NoneType:
#        date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-3]),languages=['tr']),dayfirst=True)
#        if type(date2) == NoneType:
#            date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-4]),languages=['tr']),dayfirst=True)
#            if type(date2) == NoneType:
#                date2=pd.to_datetime(dateparser.parse(tarih_duzelt(a[-5]),languages=['tr']),dayfirst=True)
#                if type(date2) == NoneType:
#                    date2=np.nan
rega_tarihi=date2
print(rega_tarihi)
    

1990-04-25 00:00:00


In [144]:
#a=resmi_gazete.data_text[537].lower().replace(' ','').split('sayi')[0].split('\n')
#while '' in a:
#    a.remove('')
#date=a[-2]
#pd.to_datetime(dateparser.parse(tarih_duzelt(date),languages=['tr']),dayfirst=True)

### Tarihler-Validasyon

##### Rega Tarihi

In [123]:
#Kanun
(df[df.kategori=='Kanun'].rega_tarihi==df[df.kategori=='Kanun'].rega_tarihi_pred).value_counts()

True    514
dtype: int64

In [124]:
#Kanun Hükmünde Kararname
(df[df.kategori=='Kanun Hükmünde Kararname'].rega_tarihi==df[df.kategori=='Kanun Hükmünde Kararname'].rega_tarihi_pred).value_counts()

True    91
dtype: int64

In [17]:
#Resmi Gazete
(df[df.kategori=='Resmi Gazete'].rega_tarihi==df[df.kategori=='Resmi Gazete'].rega_tarihi_pred).value_counts()

True     369
False    177
dtype: int64

In [125]:
#Cumhurbaşkanlığı Kararnamesi
(df[df.kategori=='Cumhurbaşkanlığı Kararnamesi'].rega_tarihi==df[df.kategori=='Cumhurbaşkanlığı Kararnamesi'].rega_tarihi_pred).value_counts()

True    54
dtype: int64

In [126]:
#Tüzük
(df[df.kategori=='Tüzük'].rega_tarihi==df[df.kategori=='Tüzük'].rega_tarihi_pred).value_counts()

True    82
dtype: int64

In [127]:
#Yönetmelik
(df[df.kategori=='Yönetmelik'].rega_tarihi==df[df.kategori=='Yönetmelik'].rega_tarihi_pred).value_counts()

True    597
dtype: int64

In [128]:
#Tebliğ
(df[df.kategori=='Tebliğ'].rega_tarihi==df[df.kategori=='Tebliğ'].rega_tarihi_pred).value_counts()

True    623
dtype: int64

##### Mevzuat Tarihi

In [132]:
#Kanun
(df[df.kategori=='Kanun'].mevzuat_tarihi==df[df.kategori=='Kanun'].mevzuat_tarihi_pred).value_counts()

True    514
dtype: int64

In [133]:
#Kanun Hükmünde Kararname
(df[df.kategori=='Kanun Hükmünde Kararname'].mevzuat_tarihi==df[df.kategori=='Kanun Hükmünde Kararname'].mevzuat_tarihi_pred).value_counts()

True    91
dtype: int64

In [135]:
#Genelge
(df[df.kategori=='Genelge'].mevzuat_tarihi==df[df.kategori=='Genelge'].mevzuat_tarihi_pred).value_counts()

True     248
False    137
dtype: int64

In [136]:
#Cumhurbaşkanlığı Kararnamesi
(df[df.kategori=='Cumhurbaşkanlığı Kararnamesi'].mevzuat_tarihi==df[df.kategori=='Cumhurbaşkanlığı Kararnamesi'].mevzuat_tarihi_pred).value_counts()

True     47
False     7
dtype: int64

In [137]:
#Tüzük
(df[df.kategori=='Tüzük'].mevzuat_tarihi==df[df.kategori=='Tüzük'].mevzuat_tarihi_pred).value_counts()

True    82
dtype: int64

In [138]:
#Özelge
(df[df.kategori=='Özelge'].mevzuat_tarihi==df[df.kategori=='Özelge'].mevzuat_tarihi_pred).value_counts()

True     741
False      9
dtype: int64

In [96]:
class RegaNoExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "rega_no"
        self.cols = constants.COLS_REGA_NO

    def iter_pattern(string, pattern, group=2):
        collection = re.finditer(pattern, string)

        if not collection:
            raise Exception(f"The pattern {pattern} isn't found in {string}")

        for match in collection:
            try:
                match_ = match.group(group)
            except Exception as e:
                raise e
            else:
                break
            
        return match_

    def _extractor_func(self, row_data):
        pattern = "(sayı:)(\d+)"

        std_txt = row_data.data_text.lower()\
                                     .replace("\n+", "***")\
                                     .replace("\s+", "")\
                                     .replace("sayi", "sayı")\
                                     .replace("sayısı", "sayı")\
                                     .replace("sayılı", "sayı")\
                                     .replace("sayıfa", "sayfa")
        try:
            rega_no = int(self.iter_pattern(std_txt, pattern, 2))#int(re.search(pattern, row).group(2))
        except Exception as e:
            try:
                pattern = "(\d+)(sayı)"
                rega_no = int(self.iter_pattern(std_txt, pattern, 1))#int(re.search(pattern, row).group(1))
            except Exception as e:
                #print(std_txt, e)
                rega_no = None
        return rega_no

class MukerrerNoExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "mukerrer_no"
        self.cols = constants.COLS_MUKERRER_NO

    def iter_pattern(string, pattern, group=2):
        collection = re.finditer(pattern, string)

        if not collection:
            raise Exception(f"The pattern {pattern} isn't found in {string}")

        for match in collection:
            try:
                match_ = match.group(group)
            except Exception as e:
                raise e
            else:
                break
            
        return match_

    def _extractor_func(self, row_data):
        pattern = "(\d+)\. mükerrer"
        std_txt = row_data.data_text.lower()
        try:
            mukerrer_no = int(self.iter_pattern(std_txt, pattern, 1))#int(re.search(pattern, row).group(2))
        except Exception as e:
            #print(std_txt, e)
            mukerrer_no = 0

        return mukerrer_no



class SiraNoExtractor(FeatureExtractor):
    def __init__(self):
        super().__init__()
        self.feature_name = "sira_no"
        self.cols = constants.COLS_SIRA_NO

    def iter_pattern(string, pattern, group=2):
        collection = re.finditer(pattern, string)

        if not collection:
            raise Exception(f"The pattern {pattern} isn't found in {string}")

        for match in collection:
            try:
                match_ = match.group(group)
            except Exception as e:
                raise e
            else:
                break
            
        return match_

    def _extractor_func(self, row_data):
        pattern = "(\d+) (e \| nci|e \| ci|e ] nei|e 1 nci|e 1 nc|e|a) (ek|ilâve|ilave)"
        std_txt=row_data.data_text.lower()\
                    .replace("'", " ")\
                    .replace("©", "ek")\
                    .replace("\(c\)", "ek")\
                    .replace("ek", " ek")\
                    .replace(":", "")\
                    .replace({  "birinci": "1",
                                "ikinci": "2",
                                "ücüncü": "3",
                                "üçüncü": "3",
                                "dördüncü": "4",
                                "besinci": "5",
                                "beşinci": "5",
                                "altıncı": "6",
                                "yedinci": "7",
                                "sekizinci": "8",
                                "dokuzuncu": "9"
                                }, regex=True)\
                    .replace("\s+", " ")\
                    .replace("s ira", "sira")\
                    .replace("slra", "sira")\
                    .replace("ı", "i")\
                    .replace("y", "")
        try:
            sira_no = str(int(self.iter_pattern(std_txt, pattern, 1))) + " ek 1"#int(re.search(pattern, row).group(2))
        except Exception as e:
            pattern = "(\d+) (ek|e+k|e|a) (\d+)"
            try:
                sira_no = str(int(self.iter_pattern(std_txt, pattern, 1))) + " ek " + str(int(self.iter_pattern(std_txt, pattern, 3)))
            except Exception as e:
                pattern = "(s\.|sira) saisi (\d+)"
                try:
                    sira_no = str(int(self.iter_pattern(std_txt, pattern, 2)))
                except Exception as e:
                    sira_no = 0
                else:
                    pass
            else:
                pass

        return sira_no



In [91]:
rega_no_extractor = RegaNoExtractor()
rega_no_extractor.extract(df)

'rega_no' cikariliyor... -> 'rega_no_pred'


100%|████████████████████████████████████████████████████████████████████████████| 2507/2507 [00:00<00:00, 6129.84it/s]


In [92]:
mukerrer_no_extractor = MukerrerNoExtractor()
mukerrer_no_extractor.extract(df)

'mukerrer_no' cikariliyor... -> 'mukerrer_no_pred'


100%|████████████████████████████████████████████████████████████████████████████| 2507/2507 [00:00<00:00, 9831.86it/s]


In [97]:
sira_no_extractor = SiraNoExtractor()
sira_no_extractor.extract(df)

'sira_no' cikariliyor... -> 'sira_no_pred'


  0%|▏                                                                               | 1/500 [00:00<00:00, 1009.46it/s]


TypeError: str.replace() takes no keyword arguments

In [ ]:
SiraNoExtractor

In [98]:
calculate_accuracy(input_df=df,
                   feature_extractor=rega_no_extractor,
                   include_na=True
                  )

Veri Sayisi: 2507
Dogru Tahmin Sayisi: 0
Yanlis Tahmin Sayisi: 2507
Basari Orani: 0.0


In [99]:
calculate_accuracy(input_df=df,
                   feature_extractor=mukerrer_no_extractor,
                   include_na=True
                  )

Veri Sayisi: 2507
Dogru Tahmin Sayisi: 2346
Yanlis Tahmin Sayisi: 161
Basari Orani: 0.9358


In [ ]:
calculate_accuracy(input_df=df,
                   feature_extractor=mukerrer_no_extractor,
                   include_na=True
                  )